In [ ]:
import numpy as np
import ipywidgets as widgets
from scipy.optimize import minimize

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import Arrow,OpenHead, NormalHead, VeeHead
output_notebook()



## scope

This notebook demonstrates some simple calculations about polarization optics.
Polarisation states and optical elements are represented in [Jones notation](https://en.wikipedia.org/wiki/Jones_calculus)

## define some polarization states in Jones notation

In [ ]:
class pstates():
    def __init__(self):
        """sime common polarization states"""
        self.H = np.array([1,0]) 
        self.V = np.array([0,1])
        self.Lin45 = np.array([1,1]) / np.sqrt(2)
        self.L = np.array([1, 1.0j]) / np.sqrt(2)
        self.R = np.array([1, -1.0j]) / np.sqrt(2)
    def randEll(self):
        """return random elliptical state"""
        p1 = (np.random.rand(4)-0.5) * 2
        sr = np.array([p1[0]+1.0j*p1[1], p1[2]+1.0j*p1[3]])
        sr = sr / np.sqrt(np.sum(np.abs(sr)**2))
        return sr

PS = pstates()

## define some optical elements

In [ ]:
class pelements():
    def __init__(self):
        self.hwp0 = [[-1.0j,0],[0,1.0j]]   # half-wave-plate, zero orientation
        self.qwp0 = 1./np.sqrt(2) * np.array([[1-1.0j,0],[0,1+1.0j]]) # quarter-wave-plate, zero orientation
        self.pol0 = np.array([[1,0],[0,0]]) # # linear polarizer, zero orientation
        
    def rotation_matrix(self, angle):
        """Rotation matrix to calculate matrix of some rotated element"""
        return np.array([[np.cos(angle),-np.sin(angle)],[np.sin(angle), np.cos(angle)]]),\
               np.array([[np.cos(-angle),-np.sin(-angle)],[np.sin(-angle), np.cos(-angle)]])
    
    def hwp(self, angle):
        """Jones matrix: Half-wave-plate, rotated by angle"""
        R1, R2 = self.rotation_matrix(angle)
        return np.dot( R1 , np.dot (self.hwp0, R2))
    
    def qwp(self, angle):
        """Jones matrix: Quarter-wave-plate, rotated by angle"""
        R1, R2 = self.rotation_matrix(angle)
        return np.dot( R1 , np.dot (self.qwp0, R2))
    
    def pol(self, angle):
        """Jones matrix: Linear polarizer, rotated by angle"""
        R1, R2 = self.rotation_matrix(angle)
        return np.dot( R1 , np.dot (self.pol0, R2))    
    

## some helper functions for plotting

In [ ]:
def statestr(v):
    return "[%.2f + %.2fi,  %.2f + %.2fi]"%(np.real(v[0]), np.imag(v[0]),np.real(v[1]), np.imag(v[1]))

def samplexy(vec, tmin=0, tmax = 1.9 * np.pi, pts = 24):
    """return Ex, Ey samples for a given input state"""
    t = np.linspace(tmin, tmax, pts)
    ex = np.real(vec[0]*np.exp(1.0j * t))
    ey = np.real(vec[1]*np.exp(1.0j * t))
    return ex, ey, t

def bk_state_plot(s, title=""):
    """plot Ey against Ex for a given input state s"""
    ex, ey, t = samplexy(s)
    s = figure(width=200, height=200, x_range=[-1.1,1.1], y_range=[-1.1,1.1],
              title=title, x_axis_label='Ex', y_axis_label="Ey")
    p = s.line(list(ex), list(ey),)# size=10, color='firebrick', alpha=0.4)
    p0 = s.circle(ex[0:1], ey[0:1], size=5, color='blue')    
    return s, p, p0

def bk_xy_plot(s, title=""):
    """plot Ex(t) and Ey(t) for a given input state"""
    ex, ey, t = samplexy(s, tmin=0, tmax=4*np.pi, pts=100)
    s = figure(width=200, height=200, x_range=[0, 2], y_range=[-1,1],
          title=title, x_axis_label='t*2pi f', y_axis_label="E")
    e1 = s.line(t/2/np.pi, ex,  legend="Ex", line_width=2)
    e2 = s.line(t/2/np.pi, ey, legend="Ey", line_width=2,line_color="red")  
    s.legend.label_text_font_size = '8pt'
    s.legend.background_fill_alpha=0.9
    s.legend.padding=1
    return s, e1, e2

def bk_update(dd, dd0, dx, dy, state):
    xx, yy, tmp = samplexy(state)
    dd.data_source.data={'x':xx, 'y':yy}
    dd0.data_source.data ={'x':xx[0:1], 'y':yy[0:1]}
    e1, e2, t = samplexy(state, tmin=0, tmax=4*np.pi, pts=100)
    dx.data_source.data={'x':t/2/np.pi, 'y':e1}
    dy.data_source.data ={'x':t/2/np.pi, 'y':e2}  
    
   

# widget: try the effect of Polarizers, HWP and QWP on different input states

In [ ]:
RandEllipState = PS.randEll()

def elem_action(key_in, element, angle):
    angle = angle/180 * np.pi
    PS = pstates()
    PE = pelements()
    instates = {'H':PS.H, 'V':PS.V, 'lin45':PS.Lin45, 'RCirc':PS.R, 'LCirc':PS.L, 'RandEllip':RandEllipState}
    elements = {'pol':PE.pol, 'qwp':PE.qwp, 'hwp':PE.hwp}
    instate = instates[key_in]
    outstate = np.dot( elements[element](angle) , instate)
    bk_update(d1,d10,ex1, ey1, instate)
    bk_update(d2,d20,ex2, ey2,  outstate)
    t1.value = "In: %s\nOut %s"%(statestr(instate), statestr(outstate))
    push_notebook()

t1 = widgets.Textarea(value="In: %s\nOut %s"%(statestr(PS.H), statestr(PS.H)))
display(t1)    

s1, d1, d10 = bk_state_plot(PS.H, title='in')
s2, d2, d20 = bk_state_plot(PS.H, title='out')
s1b, ex1, ey1 = bk_xy_plot(PS.H, title='in')
s2b, ex2, ey2 = bk_xy_plot(PS.H, title='out')
p = gridplot([[s1, s2],[s1b, s2b]], toolbar_location=None)
show(p, notebook_handle=True)


widgets.interact(elem_action, 
                 key_in=widgets.Dropdown(options=['H', 'V', 'lin45', 'RCirc', 'LCirc','RandEllip'],
                                        description='Input State'), 
                 element=widgets.Dropdown(options=['pol','hwp','qwp'],
                                         description='Element'),
                 angle = widgets.IntSlider(min=-180,max=180, description='Angle (deg)'), step=5)


## Polarization Transform: Quarter-Half-Quarter waveplate setup

In the Jones formalism, every possible state can be represented as an ellipse in the Ex-Ey plane. This includes the linear polarized state (degenerated ellipse) and circular polarization. Every of these states can be transformed into every other by passing through a setup of Quarter-Half-Quarter waveplates with suitable rotations.

In [ ]:
PE = pelements()
PS = pstates()
    
def qhq(a1,a2,a3,instate):
    """Apply action of quarter - half - quater waveplate (with angles a1, a2, a3) on instate"""
    return np.dot(PE.qwp(a3), np.dot(PE.hwp(a2), np.dot(PE.qwp(a1), instate)))

def elem_action2(key_in, a1, a2, a3):
    a1t = a1/180 * np.pi; a2t = a2/180 * np.pi; a3t = a3/180 * np.pi
    instates = {'H':PS.H, 'V':PS.V, 'lin45':PS.Lin45, 'RCirc':PS.R, 'LCirc':PS.L, 'RandEllip':RandEllipState}
    instate = instates[key_in]
    outstate = qhq(a1t, a2t, a3t, instate)
    bk_update(d3, d30, ex3, ey3,instate)
    bk_update(d4, d40, ex4, ey4, outstate)
    t2.value = "In: %s\nOut %s"%(statestr(instate), statestr(outstate))
    push_notebook()

t2 = widgets.Textarea(value="In: %s\nOut %s"%(statestr(PS.H), statestr(PS.H)))
display(t2)    
s3, d3, d30 = bk_state_plot(PS.H, title='In')
s4, d4, d40 = bk_state_plot(PS.H, title='Out')
s3b, ex3, ey3 = bk_xy_plot(PS.H, title='in')
s4b, ex4, ey4 = bk_xy_plot(PS.H, title='out')
p2 = gridplot([[s3, s4],[s3b,s4b]], toolbar_location=None)
show(p2, notebook_handle=True)

widgets.interact(elem_action2, 
                 key_in=widgets.Dropdown(options=['H', 'V', 'lin45', 'RCirc', 'LCirc','RandEllip'],
                                        description='Input State'),
                 a1 = widgets.IntSlider(min=-180,max=180, description='Angle 1 (deg)', step=5),
                 a2 = widgets.IntSlider(min=-180,max=180, description='Angle 2 (deg)', step=5),
                 a3 = widgets.IntSlider(min=-180,max=180, description='Angle 3 (deg)', step=5))

## search for 3 angles automatically

demonstrate the every-to-every transform by searching for the suitable angles using scipy.optimize

In [ ]:
def funmini(w, ein, eout):
    """function to minimize"""
    eoutnew = qhq(w[0] ,w[1], w[2], ein)
    return np.sum( np.abs(eoutnew[0]-eout[0])**2 + np.abs(eoutnew[1]-eout[1])**2 )

def findrotations(ein, eout, maxiter=10):
    """find rotation angles of the Q-H-Q setup to transform ein into eout"""
    #x0 = np.array([0,0,0])  gets stuck at local minimum
    x1 = np.array([0,.1,0])
    res = minimize(funmini, x1, args=(ein,eout))
    #print(funmini(res['x'], ein, eout))
    if not res['success']:
        print("no success!")
        return False, np.array([0,0,0])
    else:
        x = res['x']
        return True, x
    
ein = PS.V#PS.randEll()
eout = PS.R
succ, angles = findrotations(ein, eout)
s5, d5, d50 = bk_state_plot(ein, title='In')
s6, d6, d60 = bk_state_plot(eout, title='Target')
s7, d7, d70 = bk_state_plot(qhq(angles[0], angles[1], angles[2], ein), title='Out (%.0f, %.0f, %.0f)'%(
    angles[0]*180/np.pi,angles[1]*180/np.pi,angles[2]*180/np.pi))
p3 = gridplot([[s5, s6, s7]], toolbar_location=None)
show(p3)

## check conversion of various input states into output states

In [ ]:
def autofindplot(ein, eout):
    succ, angles = findrotations(ein, eout)
    s5, d5, d50 = bk_state_plot(ein, title='In')
    s6, d6, d60 = bk_state_plot(eout, title='Target')
    s7, d7, d70 = bk_state_plot(qhq(angles[0], angles[1], angles[2], ein), title='Out (%.0f, %.0f, %.0f)'%(
        angles[0]*180/np.pi,angles[1]*180/np.pi,angles[2]*180/np.pi))
    p3 = gridplot([[s5, s6, s7]], toolbar_location=None)
    show(p3)

In [ ]:
instates = {'H':PS.H, 'V':PS.V, 'lin45':PS.Lin45, 'RCirc':PS.R, 'LCirc':PS.L, 'RandEllip':RandEllipState}
for k1 in instates.keys():
    for k2 in instates.keys():
        autofindplot(instates[k1], instates[k2])